# LangGraph Simple Agent

This notebook demonstrates how to create a basic agent using LangGraph, focusing on LangGraph's state management and workflow capabilities that distinguish it from LangChain.

## Setup and Installation

First, let's install the required packages:

In [ ]:
# Install required packages
!pip install -U langgraph langchain openai python-dotenv

# Verify installations
import importlib

def check_package(package_name):
    try:
        importlib.import_module(package_name)
        return True
    except ImportError:
        return False

packages = {
    'langgraph': 'LangGraph',
    'openai': 'OpenAI',
    'dotenv': 'python-dotenv',
    'langchain': 'LangChain'
}

all_installed = True
for package, display_name in packages.items():
    installed = check_package(package)
    print(f"{display_name}: {'✅ Installed' if installed else '❌ Not installed'}")
    all_installed = all_installed and installed

if all_installed:
    print("\n✅ All required packages are installed!")
else:
    print("\n⚠️ Some packages are missing. Run the installation command again.")

## Environment Setup

Load environment variables from the `.env` file:

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API keys from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Set environment variables
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY or ""

# Verify API keys are set
if not OPENAI_API_KEY:
    print("⚠️ Warning: OPENAI_API_KEY is not set in .env file")
else:
    print("✅ API keys are set")

## LangGraph Basics

LangGraph is a framework built on top of LangChain for creating stateful, multi-agent workflows with LLMs. Unlike basic LangChain agents, LangGraph provides:

- **Explicit state management** - Track state across multiple calls
- **Conditional workflow branching** - Send execution down different paths based on state
- **Feedback loops** - Allow cycles in your agent workflow

## Initialize Language Model and Define Tools

First, let's set up the language model and define the tools our agent will use:

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool

# Initialize the language model
llm = ChatOpenAI(model="gpt-3.5-turbo")

# Define simple writing tools
def write_blog(topic):
    """Write a blog post about a given topic."""
    response = llm.invoke(f"Write a detailed and engaging blog post about: {topic}")
    return response.content

def summarize_text(text):
    """Summarize text into bullet points."""
    response = llm.invoke(f"Summarize this into bullet points:\n\n{text}")
    return response.content

def generate_title(topic):
    """Generate a catchy title and intro for a topic."""
    response = llm.invoke(f"Create a blog post title and intro paragraph for: {topic}")
    return response.content

# Create tool objects (we'll use these in our LangGraph nodes)
tools = {
    "write_blog": write_blog,
    "summarize_text": summarize_text,
    "generate_title": generate_title
}

## Define State Schema

A key aspect of LangGraph is explicit state management. A schema is a structured representation of the state, defining the expected fields and their types. 

Let's define it:

In [ ]:
from typing import Dict, List, TypedDict, Optional
from langgraph.graph import StateGraph, END

# Define our state schema
class AgentState(TypedDict):
    query: str
    tool: Optional[str]
    result: Optional[str]
    final_output: Optional[str]


# Define the initial state creation function. This function will be call the whole workflow instead of the individual nodes later.
def create_initial_state(query: str) -> AgentState:
    """Create the initial state for the workflow."""
    return {
        "query": query,
        "tool": None,
        "result": None,
        "final_output": None
    }

## Define the Nodes

LangGraph works with nodes (functions that process state). Let's define our nodes:

In [ ]:
# Tool selector node
def select_tool(state: AgentState) -> AgentState: #thanks to the state given by create_initial_state, this node will select the tool to be used.
    """Select a tool based on the query."""
    query = state["query"].lower()
    if "summary" in query or "summarize" in query:
        selected_tool = "summarize_text"
    elif "blog" in query in query:
        selected_tool = "write_blog"
    elif "title" in query:
        selected_tool = "generate_title"
    else:
        selected_tool = "write_blog"  # Default tool
    print(f"Selected tool: {selected_tool}")
    return {**state, "tool": selected_tool}

# Tool execution node
def run_tool(state: AgentState) -> AgentState:
    """Execute the selected tool."""
    tool_name = state["tool"]
    query = state["query"]
    tool_function = tools.get(tool_name)
    if not tool_function:
        result = f"Error: Tool '{tool_name}' not found."
    else:
        result = tool_function(query)
    print(f"Tool result: {result[:100]}...")
    return {**state, "result": result}


# Final output node
def format_output(state: AgentState) -> AgentState:
    """Format the final output and update the state."""
    formatted_output = f"## Tool Used: {state['tool']}\n\n{state['result']}"
    print(f"Formatted output: {formatted_output[:100]}...")
    return {**state, "final_output": formatted_output}


## Create the Workflow

Now we'll assemble these nodes into a LangGraph workflow:

In [ ]:
# Build LangGraph Workflow (each function become a node, a step in our workflow)
graph = StateGraph(AgentState)
graph.add_node("select_tool", select_tool)
graph.add_node("run_tool", run_tool)
graph.add_node("format_output", format_output)

graph.set_entry_point("select_tool")
graph.add_edge("select_tool", "run_tool")
graph.add_edge("run_tool", "format_output")
graph.set_finish_point("format_output")

tool_agent = graph.compile()


## Testing the Workflow

Let's run the workflow with a specific query:

In [ ]:
# Execute the workflow with a sample query
query = "generate a summary about the benefits of drinking water"
print(f"Running workflow for query: \"{query}\"")
initial_state = create_initial_state(query) # Create the initial state, this will be the input for the workflow
result = tool_agent.invoke(initial_state) # Execute the workflow with the initial state

# Display the results

print("====WORKFLOW RESULTS====")
print(f"\nQuery: {result['query']}")
print(f"\nSelected Tool: {result['tool']}")
print(f"\nFinal Output:\n{result['final_output']}")
print("\n" + "="*50)

## Conclusion

This notebook demonstrated how to build a simple agent using LangGraph's distinctive features:

1. **State Management** - Using TypedDict for structured state control
2. **Node-Based Processing** - Individual functions that transform state
3. **Explicit Workflows** - Clear definition of how nodes connect

Key differences from LangChain agents:
- LangGraph provides explicit state management between steps
- Workflows are defined as graphs with clear execution paths
- Each node is responsible for handling and updating specific parts of the state

This simple example shows the core concepts of LangGraph; in more complex scenarios, you could extend this with conditional branching and feedback loops.